# Word generation with N-Grams

## 1. Introduction, tool functions and parameters

In this "lab", we will be trying to generate words.  
To do that, we will use the following simple idea: if we get a list of characters, let us say "welcom", then we can try to deduct what the next character will be.  
To do that, you just have to take a dataset of words/sentences, search for all occurrences of the string "welcom", and look at the next character.  
We might for instance find $5$ occurrences of "welcom", and see that it was followed by "e" $4$ times (as in "welcome"), and by "i" once (as in welcoming).  
If our dataset is large enough, we can estimate the probabilities for the next character given a sequence of characters (here in our example the probability that the next character is an "e" would be around $80~\%$ and the probability that it is an "i" would be around $20~\%$).  
We can then start with a sequence of characters and keep appending the most likely character to form words that are likely to exist.

We start by importing some tools that we'll need.  
If you need to install `nltk`, use the following command: `conda install -c anaconda nltk`  
If you need to install `seaborn`, use the following command: `conda install -c anaconda seaborn` 

In [ ]:
import numpy as np
import itertools

import nltk
words = nltk.download('words')
from nltk.corpus import words

from PIL import Image
import matplotlib.pyplot as plt

Here we are importing the dataset of words that we will use to train our model.   
- Use list comprehension to create a (numpy) array that contains all words in `all_words` that have a length greater or equal to $2$.
- Shuffle this array.

In [ ]:
all_words = words.words()
words_list = np.array([x for x in all_words if len(x) >= 2]) #THIS LINE SHOULD BE ...
np.random.shuffle(words_list) #THIS LINE SHOULD BE ...

`asvoid` is an utilitary function, do not worry too much about it.

In [ ]:
def asvoid(arr):
    arr = np.ascontiguousarray(arr)
    return arr.view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[-1])))

Here, `arr` and `x` are basically just two 1D arrays.  
Complete the `find_index` function so that it returns the index of the first element identical in both lists (hint: use `np.nonzero`). 

In [ ]:
def find_index(arr, x):
    arr_as1d = asvoid(arr)
    x = asvoid(x)
    return np.nonzero(arr_as1d == x)[0][0] #THIS LINE SHOULD BE ...

To move closer to our goal of generating words, we will need a little bit of vocabulary.  
- A *token* is the basic unit that we are trying to predict. For example, we can try to predict the next word, or the next character.  
- A *tokenizer* is a function that splits a string into tokens. For instance, in our example we said that a token is a character, so our tokenizer will be a function that splits a string into a list of characters.  

Define a tokenizer that splits a string into a list of characters.

In [ ]:
def tokenizer(s:str):
    l = [c for c in s] #THIS LINE SHOULD BE ...
    return l

Then, we need to define some parameters for our model.  
The first parameter that we need is the number $n$ of past characters we use to predict the next one.  
In our starting example, we used $6$ characters ("welcom") to predict the $7^{\text{th}}$. If we instead picked $n=4$, we would have used only the last $4$ characters ("lcom") to predict the next one.  

Then we need an *alphabet*: the list of valid tokens (so the list of valid characters in our case).  

If we get the string "hello", what is the most likely next character? Valid answers are probably "a space" or "none". We will create a special token (a special character) to represent both, called an `end_token`. It will allow our model to end itself the words it will generate, without having to specify a desired length.   

In [ ]:
n = 3
end_token = '\x00'
alphabet = [c for c in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ-"]
alphabet.append(end_token)

## 2. The core of our model

Now, it is time to implement the core of our model.

The idea is the following:
- We take a bunch of words.
- We look at all $n+1$ characters subsets of each word.
- We count how often a given subset appears.

But first, we need to do a little bit of formatting on our input words.  

Given a word $w$, we need to split it into a list of characters using the tokenizer we defined above, and we need to append an `end_token` to it.  
But that's not it yet. We will also prepend $n$ `end_token` to each word: by doing so, our model will also be able to learn what are the most likely first characters.

Implement the formatting function to do so.

In [ ]:
def formatting(w):
    return [end_token]*n + tokenizer(w) + [end_token] #THIS LINE SHOULD BE ...

In order to count how often $n+1$ characters subsets (also called $n+1$ gram) appear, it might be a good idea to have a counter for each of those $n+1$ gram.

To do so, we first take all combinations of $n+1$ characters, and then it is your job to create a counter for each combination.  

In [ ]:
combinations = np.array([np.array(p) for p in itertools.product(alphabet, repeat = n + 1)])
counters = np.zeros((combinations.shape[0]), dtype=np.int64) #THIS LINE SHOULD BE ...

The following function takes a list of words and should:  
- Format each word properly.
- Find all its $n+1$ grams (subsets of $n+1$ characters). Reminder: `end_token` is also a character.
- Update the counters accordingly (hint: use the function `find_index` that was implemented earlier).

In [ ]:
def update_from_list(l:list[str]): #THIS FUNCTION SHOULD BE ...
    for w in l:
        t = formatting(w)
        for i in range(n, len(t)):
            gram = np.array(t[i-n:i+1])
            index = find_index(combinations, gram)
            counters[index] += 1

Now we should train our model with the database we imported at the beginning.  
Just call `update_from_list` with a subset of the dataset (a few hundreds for instance).
It will definitely take you some time. A few minutes for a $100$ words, maybe a little bit less than half an hour for a thousand.

In [ ]:
update_from_list(words_list[:1000])

Now our model is trained, fantastic!
Before generating words, we can try to give the "probability" of a word.  

Let us consider an example. If we get the word "cat", we might know that the probability that a word starts with "c" is $0.10$, then we may know that the probability of having an "a" given that we had a "c" previously is $0.30$, and finally that the probability of having a "t" after "ca" is $0.15$.  
Thus the probability of the word is $0.10 \cdot 0.30 \cdot 0.15 = 0.0045$.

First, implement a function that returns the probability of a character given the last $n$ characters.
Hint: it should:
- Get the number of times the last $n$ characters were followed by `c`.
- Get the total number of times the last $n$ characters appeared.
- Use both numbers to compute the probability.

In [ ]:
def single_probability(c, gram): #THIS FUNCTION SHOULD BE ...
    gram_plus_c = counters[find_index(combinations, np.array(list(gram) + [c]))]
    
    index = find_index(combinations, np.array(list(gram) + [alphabet[0]]))
    all_grams = counters[index:index+len(alphabet)]
    gram_total = np.sum(counters)
    
    if gram_total == 0:
        return 0
    else:
        return gram_plus_c / gram_total

Then use this function to return the probability of a word.

In [ ]:
def probability(w:str):
    probability = 1
    formatted = formatting(w)
    for i in range(n, len(formatted)-1): #THIS LOOP SHOULD BE ...
        c = formatted[i]
        gram = np.array(formatted[i-n:i])
        probability *= single_probability(c, gram)
    return probability

Try your algorithm with various words. The smaller the dataset you trained on, the bigger the change that it outputs zero so don't be scared.  
Try with some short words (and don't forget the list of words is randomized so you don't need to start with an 'a' even if you trained your model only with the beginning of the dataset).

In [ ]:
print(probability("cat"))

## 3. Word generation

It's now time to generate words!  
The basic idea is to generate characters one after the other until the generated character is an end-token.  
But we need some sort of randomness to do so, otherwise it'll always be the same word.  
What we can do is that, for each $n$-gram (group of $n$-characters), we find the probability distribution for the next character. Then we can use this probability distribution to randomly generate the next character using the last $n$ ones.

In order to avoid recomputing many times the same probability distributions, start by computing all of them.

Hints: 
- Don't use the `single_probability` function. Instead, use some numpy operations on the counters array. 
- `np.sum`, `np.nan_to_num` and `np.reshape` might help you.
- We also give you a numpy array containing all combinations of $n$-characters. You will not need it directly, but its length might help you.

In [ ]:
n_characters = np.array([np.array(p) for p in itertools.product(alphabet, repeat = n)])

def compute_probabilities(): #THIS FUNCTION SHOULD BE ...
    probabilities = counters.reshape((len(n_characters), len(alphabet)))
    probabilities = np.nan_to_num(probabilities / probabilities.sum(axis=1, keepdims=True), 0)
    return probabilities
probabilities = compute_probabilities()

Now implement a function that generates a word. Follow these steps:
- Start your word (meaning list of characters) with $n$ `end_tokens`.
- Select the probability distribution of the last $n$ characters. (The `n_characters` array given before might help you, and don't forget that you implemented a `find_index` function).
- Put it in the `fix_p` function before using it.
- Using the fixed probability distribution of the last $n$ characters, generate the next one (using `np.random.choice`).
- Continue to do so until the generated character is an `end_token` (compare it to `np.array(end_token)`, otherwise it might not work due to numpy special encodings).
- Finally, transform your array of characters into a string and return it.

In [ ]:
def generate(): #THIS FUNCTION SHOULD BE ...
    result = [end_token]*n
    while True:
        index = find_index(n_characters, result[-n:])
        choice = np.random.choice(alphabet,p=fix_p(probabilities[index]))
        result.append(choice)
        if choice == np.array('\x00'):
            break
    
    return "".join(result)

def fix_p(p):
    if p.sum() == 0:
        p = np.ones((len(p))) / len(p)
    elif p.sum() != 1.0:
        p = p*(1./p.sum())
    return p

Now is the fun part, generate some words using the function above. :)

In [ ]:
for i in range(100):
    print(generate())

Congratulations! You implemented an entire algorithm to generate random yet syntactically coherent words! This is not ChatGPT yet, but it is a good start :)

## 4. To go further ...

Here is the link towards a more advanced implementation of the same algorithm, with a little bit more mathematical details and some cool additional features: https://github.com/josh-freeman/HPshape/blob/main/examples/ngrams.ipynb